# Fazendo a extração de dados

In [ ]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

def fetch_adds(concurso):
    url = f"https://www.megasena.com/resultados/{concurso}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')

        #pegar a arrecadação total
        winners_box = soup.find('div', class_='gen-box winners')
        if winners_box:
            arrecadacao= winners_box.find('div', class_='sub-title').get_text(strip=True)
        else:
            arrecadacao=None

        #informação final
        inners_box_2= soup.find('div', class_="inner-box elem-2")
        if inners_box_2:
            gen_boxes= inners_box_2.find_all('div',class_="gen-box")
            row_data={}
            
            for gen_box in gen_boxes:
                title=gen_box.find('div',class_="title").get_text(strip=True)
                sub_title= gen_box.find('div',class_="sub-title").get_text(strip=True)

                row_data[title]=sub_title
                row_data["arrecadação"]=arrecadacao
                row_data["Número do Concurso"]= concurso

            return [row_data]
        else:
            return {"Número do Concurso": concurso, "arrecadação": arrecadacao}
    else:
        print(f"Não foi possível estabelecer uma requisição HTTP, código de erro: {response.status_code}")
        return None
        
def fetch_table(concurso):
    url = f"https://www.megasena.com/resultados/{concurso}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')                  
        # Pegar a data do concurso
        h1_tag = soup.find("h1")
        if h1_tag:
            span_tag = h1_tag.find("span")
            data_concurso = span_tag.text.strip()
        else:
            data_concurso = None
        
        # Extrair os números sorteados
        balls = soup.find_all("li", class_="ball")
        if len(balls) == 6:
            numeros_sorteados = [ball.text.strip() for ball in balls]
        else:
            numeros_sorteados = None

        table = soup.find("table", class_="_numbers -right table-col-4 mobFormat")
        if table:
            headers = [th.text.strip() for th in table.find_all('th')]
            rows = table.find_all('tr')[1:]  # Skip header row

            data = []
            for row in rows:
                cells = row.find_all('td')
                if cells:
                    row_data = {headers[i]: cells[i].text.strip() for i in range(len(cells))}
                    row_data["Data do Concurso"] = data_concurso
                    row_data["Número do Concurso"]= concurso
                    row_data["Números Sorteados"] = ", ".join(numeros_sorteados) if numeros_sorteados else None
                    data.append(row_data)

            return data
        else:
            print("Tabela não encontrada na página.")
            return None
    else:
        print(f"Não foi possível estabelecer uma requisição HTTP, código de erro: {response.status_code}")
        return None

def fetch_ganhadores_por_estado(concurso):
    url = f"https://www.megasena.com/resultados/{concurso}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')

        table = soup.find("table", class_="_numbers -right mobFormat")
        if table:
            headers = [th.text.strip() for th in table.find_all('th')]
            rows = table.find_all('tr')[1:]  # Skip header row

            data = []
            for row in rows:
                cells = row.find_all('td')
                if cells:
                    row_data = {headers[i]: cells[i].text.strip() for i in range(len(cells))}
                    data.append(row_data)

            return data
        else:
            print("Tabela de ganhadores por estado não encontrada na página.")
            return None
    else:
        print(f"Não foi possível estabelecer uma requisição HTTP, código de erro: {response.status_code}")
        return None

def save_to_excel(data_principal, data_ganhadores,data_adicional, filename):
    df_principal = pd.DataFrame(data_principal)
    df_ganhadores = pd.DataFrame(data_ganhadores)
    df_adicional= pd.DataFrame(data_adicional)
    
    with pd.ExcelWriter(filename) as writer:
        df_principal.to_excel(writer, sheet_name="dados gerais", index=False)
        df_ganhadores.to_excel(writer, sheet_name="Ganhadores por Estado", index=False)
        df_adicional.to_excel(writer,sheet_name= "Dados isolados", index=False)
        
    
    print(f"Dados salvos no arquivo {filename}")

def main():
    all_data_principal = []
    all_data_ganhadores = []
    all_data_adicional = []
    concursos=list(range(2760,0,-1))
    for concurso in tqdm(concursos, desc="Collecting Data"):  # Intervalo de concursos em ordem decrescente
        data_principal = fetch_table(concurso)
        if data_principal:
            all_data_principal.extend(data_principal)  # Acumular dados de todos os concursos
        
        data_ganhadores = fetch_ganhadores_por_estado(concurso)
        if data_ganhadores:
            for item in data_ganhadores:
                item["Concurso"] = concurso
            all_data_ganhadores.extend(data_ganhadores)  # Acumular dados de ganhadores por estado
        data_adicional = fetch_adds(concurso)
        if data_adicional:
            all_data_adicional.extend(data_adicional)

        

    if all_data_principal and all_data_ganhadores and all_data_adicional:
        save_to_excel(all_data_principal, all_data_ganhadores,all_data_adicional ,"megasena_resultados.xlsx")

if __name__ == "__main__":
    main()


# IA training code

     --------------------------------------- 11.0/11.0 MB 43.7 MB/s eta 0:00:00
     --------------------------------------- 44.7/44.7 MB 21.8 MB/s eta 0:00:00
     -------------------------------------- 301.8/301.8 kB 6.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
